In [15]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import os

SEED = 1234
# Set seed for reproducibility
np.random.seed(SEED)
random.seed(SEED)

current_folder = globals()['_dh'][0]
data_path = os.path.join(current_folder, 'data')
csv_path = os.path.join(data_path, 'training_set.csv')
df = pd.read_csv(csv_path, header=0) # load
df = df.sample(frac=1).reset_index(drop=True) # shuffle
df.head()

,NOSE_X,NOSE_Y,NOSE_Z,LEFT_EYE_INNER_X,LEFT_EYE_INNER_Y,LEFT_EYE_INNER_Z,LEFT_EYE_X,LEFT_EYE_Y,LEFT_EYE_Z,LEFT_EYE_OUTER_X,...,RIGHT_HEEL_Z,LEFT_FOOT_INDEX_X,LEFT_FOOT_INDEX_Y,LEFT_FOOT_INDEX_Z,RIGHT_FOOT_INDEX_X,RIGHT_FOOT_INDEX_Y,RIGHT_FOOT_INDEX_Z,IS_LOW_LUNGE,IS_CRESCENT_POSE,INSTRUCTION
0,0.520230,0.635046,-0.372716,0.535121,0.587382,-0.318471,0.545458,0.587426,-0.318122,0.555094,...,-0.078819,0.559809,2.670057,-0.281307,0.436801,2.701503,-0.490008,0,1,Raise your hands
1,0.630045,0.545611,-0.543785,0.655914,0.483179,-0.517935,0.669563,0.486560,-0.517600,0.681861,...,0.366401,0.647351,2.974119,0.001294,0.505877,2.966749,-0.051269,0,1,your arms up towards the sky as you
2,0.553312,0.551331,-0.818429,0.573563,0.492558,-0.776206,0.585662,0.491515,-0.775922,0.597475,...,0.378935,0.564804,2.962438,-0.260139,0.426746,2.957223,-0.107479,1,0,Have a drink
3,0.499180,0.561680,-0.737946,0.518260,0.495814,-0.707827,0.532650,0.491957,-0.707701,0.546015,...,0.086125,0.555374,2.960517,-0.262226,0.411628,2.966175,-0.381353,1,0,Drop an elbow
4,0.578184,0.535424,-0.950630,0.597474,0.472747,-0.879421,0.611845,0.472694,-0.877219,0.625602,...,0.854012,0.602773,2.916028,0.155980,0.450033,2.930970,0.370374,1,0,Push Your Knee


In [16]:
# Define X and y matrices
X_columns = df.columns[:len(df.columns) - 1]
X = df[X_columns].values
y = df['INSTRUCTION'].values

In [17]:
print(X_columns)
print ("X: ", np.shape(X))
print ("y: ", np.shape(y))

Index(['NOSE_X', 'NOSE_Y', 'NOSE_Z', 'LEFT_EYE_INNER_X', 'LEFT_EYE_INNER_Y',
       'LEFT_EYE_INNER_Z', 'LEFT_EYE_X', 'LEFT_EYE_Y', 'LEFT_EYE_Z',
       'LEFT_EYE_OUTER_X',
       ...
       'RIGHT_HEEL_Y', 'RIGHT_HEEL_Z', 'LEFT_FOOT_INDEX_X',
       'LEFT_FOOT_INDEX_Y', 'LEFT_FOOT_INDEX_Z', 'RIGHT_FOOT_INDEX_X',
       'RIGHT_FOOT_INDEX_Y', 'RIGHT_FOOT_INDEX_Z', 'IS_LOW_LUNGE',
       'IS_CRESCENT_POSE'],
      dtype='object', length=101)
X:  (600, 101)
y:  (600,)


In [18]:
import collections
from sklearn.model_selection import train_test_split

TRAIN_SIZE = 0.70
VAL_SIZE = 0.15
TEST_SIZE = 0.15
TRAIN_COUNT = int(len(y) * 0.70)
TEST_COUNT = int(len(y) - TRAIN_COUNT)


In [19]:
def train_val_test_split(X, y, train_size, test_size):
    """Split dataset into data splits."""
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size=test_size, train_size=train_size, stratify=y)

    X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, train_size=0.5, stratify=y_)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [20]:
# Create data splits
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=X, y=y, train_size=TRAIN_COUNT, test_size=TEST_COUNT)
print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print (f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print (f"Sample point: {X_train[0]} → {y_train[0]}")

X_train: (420, 101), y_train: (420,)
X_val: (90, 101), y_val: (90,)
X_test: (90, 101), y_test: (90,)
Sample point: [ 0.51662898  0.59451157 -0.47648197  0.53160924  0.5477547  -0.3986699
  0.54344171  0.54846144 -0.39841127  0.55465221  0.55011392 -0.39807943
  0.49697709  0.55058587 -0.4067041   0.48455828  0.55340815 -0.40648952
  0.47271284  0.55578554 -0.40659609  0.56496513  0.58114612 -0.06756394
  0.45733339  0.5822652  -0.09955167  0.5383693   0.65116829 -0.36612824
  0.49537271  0.65059447 -0.37650141  0.656829    0.84885412  0.01141141
  0.37297991  0.85019201 -0.07071388  0.82548141  0.6482439  -0.30853003
  0.13249227  0.68535835 -0.56832254  0.79075795  0.3068994  -0.57139653
  0.2570056   0.33422554 -0.77595699  0.79013366  0.20868532 -0.6687066
  0.29478174  0.26130414 -0.89249784  0.76716137  0.20266846 -0.59680468
  0.31962317  0.26058161 -0.80426186  0.76131988  0.23850882 -0.56627983
  0.31440723  0.28715467 -0.76308078  0.61325294  1.63374567 -0.00724168
  0.4021478